In [ ]:
from transformers import pipeline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Load emotion classifier
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

joy_contexts = {
    "stipend": "That's awesome! A 10K stipend is a great reward for your hard work.",
    "promotion": "Congratulations on your promotion! You deserve it.",
    "vacation": "Sounds like a fun getaway! Where are you headed?",
    "birthday": "Happy birthday! Hope you're celebrating in style.",
    "achievement": "Well done! Achievements like this are worth celebrating."
}

sadness_contexts = {
    "lost job": "I'm really sorry to hear that. Losing a job can be incredibly tough.",
    "breakup": "Heartbreak is painful. I'm here if you want to talk.",
    "lonely": "Feeling alone can be hard. You're not alone right now.",
    "failed": "Failure hurts, but it doesn’t define you. Let's figure out what’s next.",
    "miss someone": "Missing someone shows how deeply you care. Want to share more?"
}

anger_contexts = {
    "delay": "That sounds frustrating. Delays can really throw things off.",
    "unfair": "Feeling treated unfairly is deeply upsetting. I hear you.",
    "argument": "Arguments can leave us shaken. Want to unpack what happened?",
    "ignored": "Being ignored hurts. You deserve to be heard.",
    "cancelled": "Plans falling through can be maddening. What happened?"
}

fear_contexts = {
    "dark": "Fear of the dark is more common than you think. You're safe here.",
    "exam": "Exams can be stressful. Let’s take it one step at a time.",
    "future": "Uncertainty about the future is natural. You're not alone in this.",
    "health": "Health worries can be overwhelming. Want to talk about it?",
    "alone": "Feeling unsafe alone is valid. I'm here with you."
}
surprise_contexts = {
    "unexpected": "That must have caught you off guard! What happened?",
    "gift": "A surprise gift? That’s exciting! Who gave it to you?",
    "visit": "Surprise visits can be delightful or chaotic. Which was it?",
    "news": "Unexpected news can shake things up. Want to share more?",
    "result": "Surprising results? Tell me what happened!"
}

neutral_contexts = {
    "hi": "Hi there! How are you feeling today?",
    "hello": "Hello! I'm here to chat whenever you're ready.",
    "okay": "Got it. Let me know if there's anything you'd like to talk about.",
    "fine": "Alright. I'm here if you want to share more.",
    "nothing": "That's okay. Sometimes silence says a lot.",
    "how to": "Let's break it down together. What exactly are you trying to solve?",
    "tackle": "I can help you tackle this. Can you share more details?",
    "handle": "Handling tough situations takes strength. I'm here to support you.",
    "solve": "Let’s work through it step by step. What’s the challenge?."

}

# Generate bot response based on emotion
def generate_bot_response(emotion, user_text):
    user_text_lower = user_text.lower()
    def match_context(context_dict):
        for keyword, reply in context_dict.items():
            if keyword in user_text_lower:
                return reply
        return None
    emotion_contexts = {
        "joy" : joy_contexts,
        "sadness": sadness_contexts,
        "anger": anger_contexts,
        "fear": fear_contexts,
        "surprsie": surprise_contexts,
        "neutral": neutral_contexts}
    if emotion in emotion_contexts:
        context_reply = match_context(emotion_contexts[emotion])
        if context_reply:
            return context_reply
        fallback_responses = {
        "joy": "That's wonderful to hear! What made you feel so happy?",
        "sadness": "I'm really sorry you're feeling this way. Want to talk more about it?",
        "anger": "I hear your frustration. Let's work through it together.",
        "fear": "That sounds scary. I'm here with you — you're safe.",
        "surprise": "Wow, that’s unexpected! Tell me more.",
        "neutral": "Thanks for sharing. I'm here to listen."
    }
    return fallback_responses.get(emotion, "I'm here for you. Let's talk.")
def analyze_emotion(text):
    result = emotion_classifier(text)[0]
    top = max(result, key=lambda x: x['score'])
    return top['label'], top['score']

# Initialize conversation log
conversation_log = []

print("🤖 Emotion-Aware Chatbot is ready. Type 'exit' to quit.\n")

# Runtime loop
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Bot: Take care! I'm always here if you need me.")
        break

    # Analyze user emotion
    emotion, score = analyze_emotion(user_input)
    timestamp = datetime.now()

    # Log user message
    conversation_log.append({
        "timestamp": timestamp,
        "sender": "user",
        "text": user_input,
        "emotion": emotion,
        "confidence": round(score, 3)
    })

    # Generate bot reply
    bot_reply = generate_bot_response(emotion, user_input)
    print(f"🧠 Emotion: {emotion} ({score:.2f})")
    print(f"Bot: {bot_reply}\n")

    # Log bot message
    conversation_log.append({
        "timestamp": datetime.now(),
        "sender": "bot",
        "text": bot_reply,
        "emotion": "neutral",
        "confidence": 1.0
    })

# Convert log to DataFrame
df = pd.DataFrame(conversation_log)

# Engagement score
user_msgs = df[df['sender'] == 'user']
engagement_score = len(user_msgs) / len(df)

# Response effectiveness
effectiveness = []
for i in range(1, len(df) - 1):
    if df.iloc[i]['sender'] == 'bot' and df.iloc[i + 1]['sender'] == 'user':
        before = df.iloc[i - 1]['emotion']
        after = df.iloc[i + 1]['emotion']
        effectiveness.append((before, after))

# Emotional pattern plot
plt.figure(figsize=(10, 4))
sns.lineplot(data=df, x=df.index, y="confidence", hue="emotion", style="sender", markers=True)
plt.title("Emotional Pattern Over Conversation")
plt.xlabel("Message Index")
plt.ylabel("Emotion Confidence")
plt.xticks(df.index, df['sender'] + ": " + df['emotion'], rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()

# Summary
print("\n📊 Conversation Summary:")
print(df[['sender', 'text', 'emotion', 'confidence']])
print(f"\n🔁 User Engagement Score: {engagement_score:.2f}")
print(f"✅ Response Effectiveness Samples: {effectiveness}")

Device set to use cpu


🤖 Emotion-Aware Chatbot is ready. Type 'exit' to quit.



You:  hi


🧠 Emotion: neutral (0.91)
Bot: Hi there! How are you feeling today?



You:  today i feel very sad


...

🧠 Emotion: sadness (0.52)
Bot: I'm really sorry you're feeling this way. Want to talk more about it?



You:  help me to tackle it


🧠 Emotion: neutral (0.79)
Bot: I can help you tackle this. Can you share more details?



You:  yes


🧠 Emotion: neutral (0.85)
Bot: Thanks for sharing. I'm here to listen.

